In [12]:
import pandas as pd

filename = "results\iclr_brazil_antag_ds_rl_dp_1\iclr_brazil_antag_ds_rl_dp.h5"

In [13]:
params = pd.read_hdf(filename, key='task_parameters')
results = pd.read_hdf(filename, key="results") 

In [14]:
params

,r_train_v_test,n_train,include_S,n_jobs,standardize,gpa_cutoff,include_R,base_path,interpolation_factor
0,0.4,60000,False,7,True,3.0,False,iclr_brazil_antag_ds_rl_dp,0.25


In [15]:
results

,original_nsf,original_acc,original_g,antagonist_acc,antagonist_g,runtime,name,tid,pid,seed
0,True,0.638801,-0.089839,0.653320,-0.052754,572.951973,QSRC,0,0,40315988
1,False,0.623421,-0.075917,0.646802,-0.063707,598.158164,QSRC,0,0,40932111
2,True,0.619241,-0.094179,0.617944,-0.090138,608.531286,QSRC,0,0,60326604
3,False,0.587396,-0.057698,0.584711,-0.056921,634.523315,QSRC,0,0,66577413
4,True,0.616678,-0.043664,0.613983,-0.013002,880.249980,QSRC,0,0,25249251


In [16]:
joint = results.merge(params, how="left", left_on="tid", right_index=True)
NSF = joint[["name", "n_train", "original_nsf"]].groupby(by=["name", "n_train"]).mean().reset_index()
joint

,original_nsf,original_acc,original_g,antagonist_acc,antagonist_g,runtime,name,tid,pid,seed,r_train_v_test,n_train,include_S,n_jobs,standardize,gpa_cutoff,include_R,base_path,interpolation_factor
0,True,0.638801,-0.089839,0.653320,-0.052754,572.951973,QSRC,0,0,40315988,0.4,60000,False,7,True,3.0,False,iclr_brazil_antag_ds_rl_dp,0.25
1,False,0.623421,-0.075917,0.646802,-0.063707,598.158164,QSRC,0,0,40932111,0.4,60000,False,7,True,3.0,False,iclr_brazil_antag_ds_rl_dp,0.25
2,True,0.619241,-0.094179,0.617944,-0.090138,608.531286,QSRC,0,0,60326604,0.4,60000,False,7,True,3.0,False,iclr_brazil_antag_ds_rl_dp,0.25
3,False,0.587396,-0.057698,0.584711,-0.056921,634.523315,QSRC,0,0,66577413,0.4,60000,False,7,True,3.0,False,iclr_brazil_antag_ds_rl_dp,0.25
4,True,0.616678,-0.043664,0.613983,-0.013002,880.249980,QSRC,0,0,25249251,0.4,60000,False,7,True,3.0,False,iclr_brazil_antag_ds_rl_dp,0.25


In [17]:
NSF

,name,n_train,original_nsf
0,QSRC,60000,0.6


In [18]:
# extract means and standard errors
solution_found = joint.loc[joint.original_nsf == False]
solution_found["original_failed"] = solution_found["original_g"] > 0
solution_found["antagonist_failed"] = solution_found["antagonist_g"] > 0
solution_found

C:\Users\Valen\AppData\Local\Temp\ipykernel_2840\3515503250.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  solution_found["original_failed"] = solution_found["original_g"] > 0
C:\Users\Valen\AppData\Local\Temp\ipykernel_2840\3515503250.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  solution_found["antagonist_failed"] = solution_found["antagonist_g"] > 0


,original_nsf,original_acc,original_g,antagonist_acc,antagonist_g,runtime,name,tid,pid,seed,...,n_train,include_S,n_jobs,standardize,gpa_cutoff,include_R,base_path,interpolation_factor,original_failed,antagonist_failed
1,False,0.623421,-0.075917,0.646802,-0.063707,598.158164,QSRC,0,0,40932111,...,60000,False,7,True,3.0,False,iclr_brazil_antag_ds_rl_dp,0.25,False,False
3,False,0.587396,-0.057698,0.584711,-0.056921,634.523315,QSRC,0,0,66577413,...,60000,False,7,True,3.0,False,iclr_brazil_antag_ds_rl_dp,0.25,False,False


In [19]:
means = solution_found[["name", "n_train", "original_acc", "antagonist_acc", "original_failed", "antagonist_failed"]].groupby(by=["name", "n_train"]).mean(numeric_only=False).add_suffix("_mean").reset_index()
ses = solution_found[["name", "n_train", "original_acc", "antagonist_acc", "original_failed", "antagonist_failed"]].groupby(by=["name", "n_train"]).sem(numeric_only=False).add_suffix("_se").reset_index()

# merge
aggregated = NSF.merge(means, how="left", on=["name", "n_train"]).merge(ses, how="left", on=["name", "n_train"])

In [20]:
ses

,name,n_train,original_acc_se,antagonist_acc_se,original_failed_se,antagonist_failed_se
0,QSRC,60000,0.018013,0.031045,0.0,0.0


In [21]:
aggregated

,name,n_train,original_nsf,original_acc_mean,antagonist_acc_mean,original_failed_mean,antagonist_failed_mean,original_acc_se,antagonist_acc_se,original_failed_se,antagonist_failed_se
0,QSRC,60000,0.6,0.605408,0.615757,0.0,0.0,0.018013,0.031045,0.0,0.0


In [22]:
aggregated["interpolation_factor"] = params["interpolation_factor"]
aggregated

,name,n_train,original_nsf,original_acc_mean,antagonist_acc_mean,original_failed_mean,antagonist_failed_mean,original_acc_se,antagonist_acc_se,original_failed_se,antagonist_failed_se,interpolation_factor
0,QSRC,60000,0.6,0.605408,0.615757,0.0,0.0,0.018013,0.031045,0.0,0.0,0.25
